## 知识库的构建
### 文档解析-->分块-->embedding-->向量数据库

In [1]:
from model import RagEmbedding, RagLLM
from doc_parse import chunk, read_and_process_excel, logger

In [2]:
import pandas as pd
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb

In [3]:
pdf_files = ['./data/zhidu_employee.pdf',
            './data/zhidu_travel.pdf']

excel_files = ['./data/zhidu_detail.xlsx']

In [4]:
r_spliter = RecursiveCharacterTextSplitter(chunk_size=128,
                                          chunk_overlap=30,
                                          separators=["\n\n",
                                                      "\n", 
                                                      ".",
                                                      "\uff0e",  # Fullwidth full stop
                                                       "\u3002",  # Ideographic full stop
                                                      ",",
                                                      "\uff0c",  # Fullwidth comma
                                                      "\u3001",  # Ideographic comma
                                                     ])

In [5]:
doc_data = []
for pdf_file_name in pdf_files:
    res = chunk(pdf_file_name, callback=logger)
    for data in res:
        content = data["content_with_weight"]
        if '<table>' not in content and len(content) > 200:
            doc_data = doc_data + r_spliter.split_text(content)
        else:
            doc_data.append(content)

OCR is running...


OCR finished.
OCR: 1.9347431138157845
preprocess
Layout analysis finished.
layouts: 2.390368826687336
preprocess
Table analysis finished.
Text merging finished
OCR is running...


OCR finished.
OCR: 0.9043905958533287
preprocess
Layout analysis finished.
layouts: 1.2087930291891098
preprocess
Table analysis finished.
Text merging finished


In [6]:
for i in doc_data:
    print(len(i), "="*10,  i)

530 ========== <table><caption>病假发放标准：</caption>
<tr><td  >病假时间</td><td  >连续工龄</td><td  >发放标准</td></tr>
<tr><td></td><td  >不满二年</td><td  >60% </td></tr>
<tr><td></td><td  >已满二年不满四年</td><td  >70% </td></tr>
<tr><td  >6 个月以内病假</td><td  >已满四年不满六年</td><td  >80% </td></tr>
<tr><td></td><td  >已满六年不满八年</td><td  >90% </td></tr>
<tr><td></td><td  >八年以上</td><td  >100% </td></tr>
<tr><td></td><td  >不满一年</td><td  >40% </td></tr>
<tr><td  >6 个月以上病假</td><td  >已满一年不满三年</td><td  >50% </td></tr>
<tr><td></td><td  >连续工龄三年以上</td><td  >60% </td></tr>
</table>
58 ========== 教职工考勤管理制度
第一节适用范围
1、本制度包括了考勤、休假、加班等方面的规定。2、本制度适用于学校全体教职员工。
7 ========== 第二节考勤规定
66 ========== 1、学校的工作时间由学校决定并公布。学校内除特殊岗位特别规定外，全体教职员工均应严格执行学校的作息时间表，不迟到、不早退、不中途离校
120 ========== 。工作时间：星期一至星期四7:55-16:55 星期五7:55-16:152、所有教职工实行考勤打卡制度，工作日内，每天需打卡两次，上午上班一次和下午下班一次。3、教职员工因故(特殊情况除外)晚到或早退，应事先履行请假手续，经批准后方可离校
59 ========== 。原则上，员工请假无论时间长短、假期形式，除急诊病假或突发事件外，一律需按照请假流程。请假，需事先在钉钉系统中提交申请
102 ========== 。请假，需事先在钉钉系统中提交申请。有效的请假流程为：（1）员工休假必须事先向部门负责人申请，将工作交接清楚

In [31]:
for excel_file_name in excel_files:
    data = read_and_process_excel(excel_file_name)
    df = pd.DataFrame(data[8:], columns=data[7])
    data_excel = df.drop(columns=df.columns[11:17])
    content = data_excel.to_markdown(index=False).replace(' ', "")
    doc_data.append(f"### 以下是中央和国家机关工作人员赴地方差旅住宿费标准明细表： \n\n {content}")

In [32]:
from langchain_core.documents import Document
import uuid
documents = []
document_ids = []
for idx, chunk in enumerate(doc_data):
    is_table = 0
    if "table" in chunk:
        is_table = 1
    if idx == len(doc_data) - 1:
        is_table = 1
    doc_id = str(uuid.uuid4())
    document = Document(
        page_content=chunk,
        metadata={"type": "ori", "is_table": is_table})
    documents.append(document)
    document_ids.append(doc_id)

In [21]:
embedding_cls = RagEmbedding()

In [33]:
doc_txts = dict(zip(document_ids, documents))

In [34]:
import pickle
with open('./data/zhidu_db.pickl', 'wb') as file:
    pickle.dump(doc_txts, file)

In [12]:
chroma_client = chromadb.HttpClient(host="localhost", port=8000)

In [13]:
embedding_db = Chroma.from_documents(documents,
                                     embedding_cls.get_embedding_fun(),
                                     client=chroma_client,
                                     collection_name="zhidu_db",
                                     )

In [14]:
query = '迟到有什么规定？'

In [15]:
related_docs = embedding_db.similarity_search(query, k=2)

In [16]:
related_docs

[Document(metadata={'is_table': 0, 'type': 'ori'}, page_content='。如遇紧急情况，口头申请请假的，应在上班后两天内办理补请假手续，未在规定时间内办理的，逾期无效，按旷工处理。4、无工作理由，超过上班时间到岗的，视为迟到；未到下班时间提前离校的，视为早退；中途未经批准离校，视为旷工；迟到、早退、旷工者按照相关办法处理'),
 Document(metadata={'is_table': 0, 'type': 'ori'}, page_content='第五节违规违纪处理\n1. 迟到、早退(1)劳动考勤是公司支付薪资的依据和职工年度岗位考核内容之一。（2）迟到或早退60分钟以上（含60 分钟），每次视同缺勤1 天。（3）职工迟到、早退、脱岗累计超过3 次的（含），从第1 次起，每次扣减工资50 元。')]

In [17]:

chroma_client = chromadb.HttpClient(host="localhost", port=8000)

def delete_chroma_collection(name, embedding_fn, chroma_client):
    collection = Chroma(name,
                        embedding_fn,
                        client=chroma_client)
    Chroma.delete_collection(collection)

In [ ]:
delete_chroma_collection("zhidu_db", embedding_cls.get_embedding_fun(), chroma_client)

## RAG问答流程

In [ ]:
llm = RagLLM()

In [ ]:
prompt_template = """
你是企业员工助手，熟悉公司考勤和报销标准等规章制度，需要根据提供的上下文信息context来回答员工的提问。\
请直接回答问题，如果上下文信息context没有和问题相关的信息，请直接回答[不知道,请咨询HR] \
问题：{question} 
"{context}"
回答：
"""

In [ ]:
def run_rag_pipline(querys, k=3):
    for query in querys:
        
        related_docs = embedding_db.similarity_search(query, k=k)
        context = "\n".join([f"上下文{i+1}: {doc.page_content} \n" \
                             for i, doc in enumerate(related_docs)])
        print()
        print()
        print("#"*100)
        print(f"query: {query}")
        print(f"context: {context}")
        llm_prompt = prompt_template.replace("{question}", query).replace("{context}", context)
        response = llm(llm_prompt, stream=True)
        print(f"response: ")
        for chunk in response:
            print(chunk.choices[0].text, end='', flush=True)

In [ ]:
run_rag_pipline(['加班有加班费吗？'])

In [ ]:
run_rag_pipline(['出差可以买意外保险吗？需要自己购买吗'])

In [ ]:
run_rag_pipline(['那个，我们公司有什么规定来着？ 您公司的具体规定需要参考您所在公司的员工手册或相关政策文件。通常，公司会有包括但不限于工作时间、请假流程、加班政策、行为准则、保密协议等方面的规章制度。建议查阅您的员工手册或向人力资源部门咨询以获取详细信息。'])

In [ ]:
run_rag_pipline(['公司派我去上海，钱怎么弄？'])

In [ ]:
run_rag_pipline(['如果我明天要出去，那个报销的事情怎么办？'])

In [ ]:
run_rag_pipline(['如果我明天要出去，那个报销的事情怎么办？ 如果您明天需要外出，关于报销的事宜，请提前与您的直接上级或财务部门沟通。确保了解公司的报销流程和所需文件，如需预支费用，应按照公司规定提交申请。同时，保留好所有相关票据，以便返回后及时完成报销手续。如果可能，可以预先填写报销单的部分内容，以加快后续处理速度。'])